In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine, MetaData, delete
import numpy as np
from scipy.stats import linregress as fit

import time
import pickle
import os
from tqdm import tqdm

from datetime import timedelta, datetime
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import utils
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
i = 21

# this list came from sessions api for 2023, "location" attribute
locations = ['Sakhir', 'Jeddah', 'Melbourne', 'Baku', 'Miami', 'Monaco',
       'Barcelona', 'Montréal', 'Spielberg', 'Silverstone', 'Budapest',
       'Spa-Francorchamps', 'Zandvoort', 'Monza', 'Marina Bay', 'Suzuka',
       'Lusail', 'Austin', 'Mexico City', 'São Paulo', 'Las Vegas',
       'Yas Island']
location = locations[i]
print(location)
year = 2023

In [ ]:
session = utils.get_session(location, year)
display(session)

if "Practice 1" in session.session_name.tolist():
    session_key_fp1 = session.query(" session_name == 'Practice 1'").session_key.iloc[0]
else:
    session_key_fp1 = session.query(" session_name == 'Qualifying'").session_key.iloc[0]
if "Practice 2" in session.session_name.tolist():
    session_key_fp2 = session.query(" session_name == 'Practice 2'").session_key.iloc[0]
else:
    session_key_fp2 = session.query(" session_name == 'Sprint'").session_key.iloc[0]

session_key_race = session.query(" session_name == 'Race'").session_key.iloc[0]
race_start_time = pd.to_datetime(session.query(" session_name == 'Race'").date_start.iloc[0])
race_end_time = pd.to_datetime(session.query(" session_name == 'Race'").date_end.iloc[0])

driver_config = {row['name_acronym']:row['driver_number'] for ind, row in utils.get_data(f'https://api.openf1.org/v1/drivers?session_key={session_key_race}').iterrows()}
driver_config_reverse = {v: k for k, v in driver_config.items()}


lap_data = pd.concat([utils.get_data(f'''https://api.openf1.org/v1/laps?session_key={session_key_fp1}'''), utils.get_data(f'''https://api.openf1.org/v1/laps?session_key={session_key_fp2}''')]).dropna(subset =['lap_duration'])
lap_data['date_start'] = pd.to_datetime(lap_data['date_start'],format="ISO8601")
lap_data['date_end'] = lap_data.apply(lambda x: x.date_start + timedelta(seconds = x.lap_duration), axis = 1)

ref_lap_distances = pd.DataFrame()
full_laps = pd.DataFrame()

LAP_THRESHOLDS = 50

print("Building model...")
for _, lap in tqdm(lap_data.sort_values(by = 'lap_duration').iloc[:LAP_THRESHOLDS].iterrows()):

    driver_number = lap.driver_number
    start_time = lap.date_start
    end_time = lap.date_end
    session_key = lap.session_key
    meeting_key = lap.meeting_key
    lap_duration = (end_time - start_time).total_seconds()

    config = {'start_time' : start_time - timedelta(seconds = 2), 'end_time' : start_time + timedelta(seconds = 2), 'session_key' : session_key, 'driver_number' : driver_number}
    car_data, location_data = utils.get_data_channels(config)

    car_data.loc[len(car_data)] = [driver_number, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, start_time, session_key, meeting_key]
    location_data.loc[len(location_data)] = [np.nan, np.nan, np.nan, driver_number, start_time, session_key, meeting_key]

    merged = utils.merge_data_channels(car_data.sort_values(by = 'date'), location_data.sort_values(by = 'date'))
    merged = utils.compute_distance(merged, start_time)
    ref_lap_distances = pd.concat([ref_lap_distances, merged[merged.date == start_time][['x','y','distance', 'driver_number']]])

    config = {'start_time' : start_time, 'end_time' : end_time, 'session_key' : session_key, 'driver_number' : driver_number}
    car_data, location_data = utils.get_data_channels(config)
    merged = utils.merge_data_channels(car_data.sort_values(by = 'date'), location_data.sort_values(by = 'date'))
    merged = utils.compute_distance(merged, start_time)
    full_laps = pd.concat([full_laps, merged[['x','y','distance', 'driver_number']]])

ref_lap_distances.dropna(inplace = True)
full_laps.dropna(inplace = True)

In [ ]:
ref_lap_distances['mse'] = 0
ref_lap_distances.reset_index(inplace = True)
# ref_lap_distances

for index, row in ref_lap_distances.iterrows():
  ref_points = [[row.x, row.y]]
  mse = mean_squared_error(ref_lap_distances[['x', 'y']], np.repeat(ref_points, len(ref_lap_distances), 0))
  ref_lap_distances.mse.loc[index] = mse

start_line = ref_lap_distances.iloc[ref_lap_distances.mse.argmin()].x.round(3), ref_lap_distances.iloc[ref_lap_distances.mse.argmin()].y.round(3)
print(start_line)

In [ ]:
data = ref_lap_distances.iloc[:]

fig = go.Figure()
fig.add_trace(go.Scatter(x = data.x, y = data.y, mode = 'markers'))
fig.show()

In [ ]:
print(start_line)

data = full_laps.iloc[:5000]

yrange = [data.y.min()-500,data.y.max()+500]
xrange = [data.x.min()-500,data.x.max()+500]

if ((yrange[1] - yrange[0])/(xrange[1] - xrange[0])) > 1:
    height = 600
    width = int(600*(yrange[1] - yrange[0])/(xrange[1] - xrange[0]))
    
else:
    width = 600
    height = int(600*(yrange[1] - yrange[0])/(xrange[1] - xrange[0]))

fig = go.Figure()
fig.add_trace(go.Scatter(x = data.x, y = data.y, mode = 'markers'))
fig.add_trace(go.Scatter(x = [start_line[0]], y = [start_line[1]], mode = 'markers', marker = {'size':15}))
fig.update_layout(width = width, height = height, yaxis_range = yrange, xaxis_range = xrange)
fig.show()

In [ ]:
before_start_line = (15, 2036)
after_start_line = (3000, 2357)

print(start_line, before_start_line, after_start_line)